In [ ]:
import os
os.chdir('..')

In [ ]:
from src.data_prep.load_data import (
    get_league_data,
    get_team_history,
    get_league_history,
    get_current_season_information,
    get_managers_information_league
)
from src.data_prep.reshape_data import (
    summarise_season_current,
    summarise_season_history
)
from src.data_prep.output_data import (
    filter_rehsaped_season_history,
    get_season_overview,
    get_current_champions,
    get_most_wins,
    get_best_rank_points,
    get_seasons_by_top_three_teams,
    get_titles_won_summary
)

In [ ]:
#Variables
season_start_year = 2001
league_id=

In [ ]:
league_data, team_data = get_league_data(league_id=league_id)
manager_information = get_managers_information_league(team_data=team_data)
season_history = get_league_history(team_data=team_data)
final_gw_finished, current_season_year, team_ids = get_current_season_information()

In [ ]:
season_current_df = summarise_season_current(
        league_data=league_data,
        team_data=team_data,
        manager_information=manager_information,
        current_season_year=current_season_year,
        team_ids=team_ids
        )

season_history_df = summarise_season_history(
        season_history=season_history,
        season_current_df=season_current_df,
        final_gw_finished=final_gw_finished)

season_history_df_filtered = filter_rehsaped_season_history(
    season_start_year=season_start_year,
    df=season_history_df
) 
season_overview = get_season_overview(
    season_history_df_filtered=season_history_df_filtered,
    manager_information=manager_information,
    team_ids=team_ids
)
current_champions = get_current_champions(
    season_history_df_filtered=season_history_df_filtered,
    season_overview=season_overview
)
most_season_won_teams_str = get_most_wins(season_overview=season_overview)
best_rank_teams_str = get_best_rank_points(
    season_history_df_filtered=season_history_df_filtered,
    column='rank'
)
best_points_teams_str = get_best_rank_points(
    season_history_df_filtered=season_history_df_filtered,
    column='total_points'
)
seasons_top_three=get_seasons_by_top_three_teams(
    season_history_df_filtered=season_history_df_filtered
)
titles_won_summary = get_titles_won_summary(
    season_overview=season_overview
)
